In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import optuna
from shap_analysis import SHAPAnalyzer, function_map
from sklearn.model_selection import train_test_split
import shap

df = pd.read_csv("thesis_cleaned_transformed_histone_dataset_categorical(H3K23me1).tsv", sep="\t")

X = df.drop(columns = ['Gene Expression (FPKM)_log'])
y = df['Gene Expression (FPKM)_log']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2]:
def objective(trial):
    # Define the hyperparameters to tune
    params = {
        'booster': 'gbtree',
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'tree_method': 'hist',
        'device': 'cuda',
        'learning_rate': trial.suggest_float("learning_rate", 0.001, 0.05),
        'max_depth': trial.suggest_int("max_depth", 10, 30),
        'subsample': trial.suggest_float("subsample", 0.4, 1.0),
        'gamma': trial.suggest_float("gamma", 0.0, 1.0),
        'colsample_bytree': trial.suggest_float("colsample_bytree", 0.5, 1.0),
        'lambda': trial.suggest_float("lambda", 1e-3, 15),
        'alpha': trial.suggest_float("alpha", 1e-3, 15),
    }

    # Prepare the DMatrix
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_test, label=y_test)

    # Train the model
    model = xgb.train(params, dtrain, num_boost_round=100, 
                      evals=[(dvalid, 'validation')],
                      early_stopping_rounds=10, 
                      verbose_eval=False)

    # Predict on validation data
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_test)


    analyzer = SHAPAnalyzer(X_test, shap_values, function_map)

    analyzer.calculate_high_value_shap_means()

    results_df, total_score, result_summary = analyzer.get_results()

    total_mismatches = result_summary.get("Mismatch", 0)
    total_matches = result_summary.get("Match", 0)
    accuracy = total_matches / (total_matches + total_mismatches) if (total_matches + total_mismatches) > 0 else 0

    # Define the objective score
    alpha = 1.0  # Weight for accuracy
    beta = 1.0   # Weight for mismatch count
    objective_score = alpha * accuracy + beta * total_score
    
    trial.set_user_attr("result_summary", result_summary)
    trial.set_user_attr("results_df", results_df)

    print(f"\n--- Trial {trial.number} ---")
    print(f"Params: {params}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Total Matches: {total_matches}, Total Mismatches: {total_mismatches}")
    print(f"Objective Score: {objective_score:.4f}")

    return objective_score

# Create a study object and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Output the best parameters
print("Best parameters:", study.best_params)
print("Best RMSE:", study.best_value)

[I 2025-03-12 18:13:56,242] A new study created in memory with name: no-name-4cbdb5ad-319e-4db2-a673-014c85a0b9fe
[18:14:03] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
[I 2025-03-12 18:14:07,182] Trial 0 finished with value: 1.6983465573271133 and parameters: {'learning_rate': 0.014715977813907235, 'max_depth': 22, 'subsample': 0.9921555906784988, 'gamma': 0.2732788339988851, 'colsample_bytree': 0.9631124447854944, 'lambda': 7.7698252060498625, 'alpha': 2.4945277557578533}. Best is trial 0 with value: 1.6983465573271133.



--- Trial 0 ---
Params: {'booster': 'gbtree', 'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist', 'device': 'cuda', 'learning_rate': 0.014715977813907235, 'max_depth': 22, 'subsample': 0.9921555906784988, 'gamma': 0.2732788339988851, 'colsample_bytree': 0.9631124447854944, 'lambda': 7.7698252060498625, 'alpha': 2.4945277557578533}
Accuracy: 0.8125
Total Matches: 13, Total Mismatches: 3
Objective Score: 1.6983


[18:14:08] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
[I 2025-03-12 18:14:09,496] Trial 1 finished with value: 2.301152271567844 and parameters: {'learning_rate': 0.02069520388540478, 'max_depth': 12, 'subsample': 0.948762259940102, 'gamma': 0.8825084265416093, 'colsample_bytree': 0.5382543624212902, 'lambda': 10.560065324004052, 'alpha': 11.760904947993541}. Best is trial 1 with value: 2.301152271567844.



--- Trial 1 ---
Params: {'booster': 'gbtree', 'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist', 'device': 'cuda', 'learning_rate': 0.02069520388540478, 'max_depth': 12, 'subsample': 0.948762259940102, 'gamma': 0.8825084265416093, 'colsample_bytree': 0.5382543624212902, 'lambda': 10.560065324004052, 'alpha': 11.760904947993541}
Accuracy: 0.8750
Total Matches: 14, Total Mismatches: 2
Objective Score: 2.3012


[18:14:11] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
[I 2025-03-12 18:14:12,051] Trial 2 finished with value: 2.229872065363452 and parameters: {'learning_rate': 0.019140323486613143, 'max_depth': 30, 'subsample': 0.4286746691175212, 'gamma': 0.9644929141008773, 'colsample_bytree': 0.5513437844559965, 'lambda': 9.063242025002944, 'alpha': 7.889030726670226}. Best is trial 1 with value: 2.301152271567844.


...


--- Trial 97 ---
Params: {'booster': 'gbtree', 'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist', 'device': 'cuda', 'learning_rate': 0.030411921647391336, 'max_depth': 21, 'subsample': 0.9520902468876217, 'gamma': 0.21987578478949363, 'colsample_bytree': 0.5515985004133551, 'lambda': 2.696123273412341, 'alpha': 9.97382343577853}
Accuracy: 0.8750
Total Matches: 14, Total Mismatches: 2
Objective Score: 2.3026


[18:23:41] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
[I 2025-03-12 18:23:44,162] Trial 98 finished with value: 2.3507472365163267 and parameters: {'learning_rate': 0.026947987948252487, 'max_depth': 21, 'subsample': 0.9163018617859224, 'gamma': 0.2940457357149802, 'colsample_bytree': 0.5356067651772336, 'lambda': 0.3528650724606863, 'alpha': 7.940071144651425}. Best is trial 74 with value: 2.3551790644414723.



--- Trial 98 ---
Params: {'booster': 'gbtree', 'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist', 'device': 'cuda', 'learning_rate': 0.026947987948252487, 'max_depth': 21, 'subsample': 0.9163018617859224, 'gamma': 0.2940457357149802, 'colsample_bytree': 0.5356067651772336, 'lambda': 0.3528650724606863, 'alpha': 7.940071144651425}
Accuracy: 0.8750
Total Matches: 14, Total Mismatches: 2
Objective Score: 2.3507


[18:23:50] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
[I 2025-03-12 18:23:53,401] Trial 99 finished with value: 2.08930490212515 and parameters: {'learning_rate': 0.042636306803579396, 'max_depth': 21, 'subsample': 0.7091829650744815, 'gamma': 0.29679140354986194, 'colsample_bytree': 0.5733902331166507, 'lambda': 0.29701753499797756, 'alpha': 7.139605281593096}. Best is trial 74 with value: 2.3551790644414723.



--- Trial 99 ---
Params: {'booster': 'gbtree', 'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist', 'device': 'cuda', 'learning_rate': 0.042636306803579396, 'max_depth': 21, 'subsample': 0.7091829650744815, 'gamma': 0.29679140354986194, 'colsample_bytree': 0.5733902331166507, 'lambda': 0.29701753499797756, 'alpha': 7.139605281593096}
Accuracy: 0.7500
Total Matches: 12, Total Mismatches: 4
Objective Score: 2.0893
Best parameters: {'learning_rate': 0.025676353387498563, 'max_depth': 17, 'subsample': 0.9917062302935468, 'gamma': 0.1692202071340728, 'colsample_bytree': 0.5089421534430815, 'lambda': 1.386350901902838, 'alpha': 12.06874097935106}
Best RMSE: 2.3551790644414723


NameError: name 'analyzer' is not defined